In [162]:
import torch
import numpy as np
from models.esvs import V_MTRN
from datasets.pickle_dataset import MultiPickleDataset
from torch.utils.data import DataLoader

from frame_sampling import RandomSampler
from torchvideo.samplers import frame_idx_to_list
from attribution.online_shapley_value_attributor import OnlineShapleyAttributor
import pandas as pd

from subset_samplers import ConstructiveRandomSampler

ImportError: cannot import name 'V_MTRN' from 'models.esvs' (/media/deepthought/SCRATCH/Tom/eve_home/C1-Action-Recognition-TSN-TRN-TSM/src/models/esvs.py)

In [161]:
device = torch.device('cuda:0')
dtype = torch.float

n_frames = 4

def no_collate(args):
    return args

frame_sampler = RandomSampler(frame_count=n_frames, snippet_length=1, test=True)

In [128]:
models = [_MTRN(frame_count=i) for i in range(1,9)]
for j, m in enumerate(models):
    models[j].load_state_dict(torch.load(f'../datasets/epic/models/mtrn-frames={j+1}.pt'))

model = models[n_frames]

In [46]:
dataset = MultiPickleDataset('../datasets/epic/features/p01_features.pkl')
dataloader = DataLoader(dataset, batch_size=1)



In [47]:
# out = torch.softmax(torch.rand((1,97)), dim=-1)
data = iter(dataloader)

inp, lab = data.next()

In [60]:
def subsample_frames(video):
    video_length = len(video)
    if video_length < n_frames:
        raise ValueError(f"Video too short to sample {n_frames} from")
    sample_idxs = np.array(frame_idx_to_list(frame_sampler.sample(video_length)))
    return sample_idxs, video[sample_idxs]


# input_ = torch.cat(inp).to(dtype=dtype)

# input_.shape
# subsample_frames(inp.squeeze())
sample_idx, sample_video = subsample_frames(inp.squeeze())

In [158]:
out = models(sample_video.to(device))

RuntimeError: shape '[-1, 1280]' is invalid for input of size 1024

In [86]:
verbs = torch.softmax(out.cpu(), dim=-1)

nouns = torch.softmax(torch.rand((1,300)), dim=-1)


result_scores = torch.cat((verbs, nouns), dim=-1)
result_scores.shape

torch.Size([1, 397])

In [121]:
class_priors = pd.read_csv('../datasets/epic/labels/verb_class_priors.csv', index_col='verb_class')['prior'].values

In [122]:
attributor = OnlineShapleyAttributor(
    single_scale_models=models,
    priors=class_priors,
    n_classes=len(class_priors),
    device=device,
    subset_sampler=ConstructiveRandomSampler(max_samples=128, device=device)
)

In [131]:
esvs, _ = attributor.explain(sample_video.to(device))

In [140]:
noun_esvs = torch.softmax(torch.rand((1,n_frames,300)), dim=-1)

In [142]:
verb_esvs = esvs.cpu().unsqueeze(0)

In [143]:
result_esvs = torch.cat((verb_esvs, noun_esvs), dim=-1)

In [145]:
result_esvs.shape

torch.Size([1, 4, 397])

In [165]:
scores.detach().item()

ValueError: only one element tensors can be converted to Python scalars

In [152]:
inp.shape[1]

195

In [155]:
lab['narration_id'] = lab['narration_id'][0]

In [156]:
lab['narration_id']

'P01_01_0'